In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder,PolynomialFeatures,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
scaler = StandardScaler(with_mean=False)
label = OneHotEncoder(handle_unknown='ignore')

target variable is 'seek help'

In [5]:
df = pd.read_csv("cleaned_survey.csv")

In [6]:
df['seek_help'].value_counts()

seek_help
No            641
Don't know    363
Yes           247
Name: count, dtype: int64

In [7]:
df=df[df['seek_help']!="Don't know"]

In [8]:
cat_col = [ 'Country', 'self_employed', 'family_history', 'treatment',
       'work_interfere', 'no_employees', 'remote_work', 'tech_company',
       'benefits', 'care_options', 'wellness_program',
       'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'gender']

num_col = ['Age']
df['seek_help']=df['seek_help'].map({'No':0,'Yes':1,"Don't know":2})

In [9]:
cat_col_trans = Pipeline([('cat',label),('scale',scaler)])
num_col_trans = Pipeline([('_scale_',scaler)])

In [10]:
preprocessor = ColumnTransformer([('categorical',cat_col_trans,cat_col),
                                  ('numerical',num_col_trans,num_col)])

In [11]:
tree = DecisionTreeClassifier()
logic = LogisticRegression(max_iter=1200)
xgb = XGBClassifier(n_estimators=1200,
    learning_rate=0.01,
    max_depth=10,
    reg_lambda=10,     
    reg_alpha=10,       
    gamma=0.8,      
    random_state=42)

In [12]:
x = df.drop(columns=['seek_help'])
y = df['seek_help']
x_train,x_test,y_train,y_test = train_test_split( x , y , train_size=0.8,random_state=42 )

In [13]:
logic_pipe = Pipeline([('preprocessor',preprocessor),('model',logic)])
logic_pipe.fit(x_train,y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numerical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
y_predict=logic_pipe.predict(x_test)
tup = (y_test,y_predict)

In [15]:
print(classification_report(*tup))
print(confusion_matrix(*tup))
accuracy_score(*tup)

              precision    recall  f1-score   support

           0       0.94      0.91      0.92       135
           1       0.74      0.81      0.78        43

    accuracy                           0.89       178
   macro avg       0.84      0.86      0.85       178
weighted avg       0.89      0.89      0.89       178

[[123  12]
 [  8  35]]


0.8876404494382022

In [16]:
xgb_pipe = Pipeline([('preprocessor',preprocessor),('model',xgb)])
xgb_pipe.fit(x_train,y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('categorical', ...), ('numerical', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
y_predict=xgb_pipe.predict(x_test)
tup = (y_test,y_predict)

In [18]:
print(classification_report(*tup))
print(confusion_matrix(*tup))
accuracy_score(*tup)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94       135
           1       0.81      0.81      0.81        43

    accuracy                           0.91       178
   macro avg       0.88      0.88      0.88       178
weighted avg       0.91      0.91      0.91       178

[[127   8]
 [  8  35]]


0.9101123595505618

In [19]:
import pickle

In [20]:
with open("classification_xgb.pkl",'wb') as f:
    pickle.dump(xgb_pipe,f)

In [21]:
with open("classification_logic.pkl",'wb') as f:
    pickle.dump(logic_pipe,f)

In [ ]:
#used this in demo to save time
col = df.columns.tolist()

with open ('temp.txt','w')as f:
    for element in col:
        f.write(f"{element} = st.selectbox('{element}',{list(df[element].unique())})\n")

In [30]:
#used this in demo to save time
col = df.columns.tolist()

with open ('temp2.txt','w')as f:
    f.write('pd.DataFrame([{')
    for element in col:
        f.write(f'"{element}" : {element},\n')
    f.write("}])")
